# Update Table and Column Tags

This notebook provides an example for updated a table and columns custom properties. To get started, you will need to [obtain an API key](https://app.secoda.co/settings/api)

Input your API key and Secoda API endpoint URL below. The URL for the cloud instance is `https://api.secoda.co`. If you are self-hosting Secoda or on the EU instance, you will have to update variable.

In [ ]:

import requests

API_KEY = "api_key"
SECODA_API_URL = "https://api.secoda.co"

session = requests.Session()
session.headers.update(dict(
    Authorization=f"Bearer {API_KEY}"
))


def build_url(url: str):
    return f"{SECODA_API_URL}{url}"

## Update table and column tags
In this step, we find the table using the title, schema, and database and then update the tags.

In [ ]:
def get_tables():
    res = session.get(build_url("/table/tables/"))
    results = res.json().get("results")
    if results and len(results) >= 1:
        return results
    return []

def get_table_columns(parent_id):
    res = session.get(build_url(f"/table/columns/?parent_id={parent_id}"))
    results = res.json().get("results")
    if results and len(results) >= 1:
        return results[0]
    return []

def update_table_tags(id, tags):
    session.patch(
        build_url(f"/table/tables/{id}/"),
        json=dict(
            tags=tags,
        ),
    )

def update_column_tags(id, tags):
    session.patch(
        build_url(f"/table/columns/{id}/"),
        json=dict(
            tags=tags,
        ),
    )

tables = get_tables()

# Convert table tags to an array if the "tags" field is not an array
for table in tables:
    tags = table.get("tags", [])
    if not isinstance(tags, list):
        tags = [tags]
        print("UPDATING TABLE TAGS: ", table.get("title"), " ", tags)
        update_table_tags(table.get("id"), tags)

    # Convert column tags to an array if the "tags" field is not an array
    columns = get_table_columns(table.get("id"))
    if columns:
        tags = columns.get("tags", [])
        for column in columns:
            if not isinstance(tags, list):
                tags = [tags]
                print("UPDATING COLUMN TAGS: ", column.get("title"), " ", tags)
                update_column_tags(column.get("id"), tags)

